In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
import gmaps.datasets
import gmaps
import time
api_key = "ENTER YOUR API KEY HERE"

# Drunk Driving Fatal Collisions vs Bars Location

In [21]:
filepath_TPS_alcohol = "Resources/Fatal_Collisions.csv"
TPS_alcohol_df = pd.read_csv(filepath_TPS_alcohol)
TPS_alcohol_df

,X,Y,Index_,ACCNUM,YEAR,DATE,TIME,Hour,STREET1,STREET2,...,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,Hood_ID,Neighbourh,ObjectId
0,-79.227590,43.816245,7377669,1275814,2012,2012-01-12T05:00:00.000Z,558,5,NEILSON RD,FINCH AVE E,...,,Yes,,Yes,,,,132,Malvern (132),1
1,-79.447742,43.637614,80000014,131058335,2013,2013-11-29T05:00:00.000Z,558,5,1491 LAKE SHORE BLVD W,,...,,,Yes,Yes,,,,85,South Parkdale (85),2
2,-79.259883,43.716773,80944448,8001077165,2018,2018-06-14T04:00:00.000Z,558,5,ST CLAIR AVE E,DANFORTH RD,...,,,,Yes,,,,120,Clairlea-Birchmount (120),3
3,-79.316890,43.821345,6253540,1162012,2010,2010-04-13T04:00:00.000Z,559,5,STEELES AVE E,BIRCHMOUNT RD,...,,Yes,,Yes,Yes,,Yes,116,Steeles (116),4
4,-79.574641,43.728129,80501551,4003267619,2014,2014-11-09T05:00:00.000Z,559,5,KIPLING AVE,WESTHUMBER BLVD,...,,Yes,,Yes,Yes,,,4,Rexdale-Kipling (4),5
5,-79.513431,43.758048,8000170,143339762,2014,2014-11-21T05:00:00.000Z,600,6,FINCH AVE W,DRIFTWOOD AVE,...,,,,,,,,25,Glenfield-Jane Heights (25),6
6,-79.263774,43.801421,80493406,5002013764,2015,2015-11-24T05:00:00.000Z,603,6,MCCOWAN RD,KENHATCH BLVD,...,,,Yes,Yes,Yes,,,129,Agincourt North (129),7
7,-79.437390,43.652745,6665360,1249510,2011,2011-08-16T04:00:00.000Z,606,6,374 BROCK AVE,PUBLIC LANE,...,,,,,,,,83,Dufferin Grove (83),8
8,-79.587123,43.699229,80000029,142120007,2014,2014-05-21T04:00:00.000Z,610,6,HIGHWAY 27 S,BELFIELD RD,...,,,,,,,,1,West Humber-Clairville (1),9
9,-79.326206,43.664164,80489505,5001612624,2015,2015-09-18T04:00:00.000Z,610,6,QUEEN ST E,HILTZ AVE,...,,,,,,,,65,Greenwood-Coxwell (65),10


In [22]:
lats=[]
lngs=[]
for index,row in TPS_alcohol_df.iterrows():
    if row["ALCOHOL"] == "Yes": 
        lats.append(row[0])
        lngs.append(row[1])

In [23]:
drunk_drivers = {"longitudes":lngs, "latitudes":lats}
drunk_drivers_df = pd.DataFrame(drunk_drivers)
drunk_drivers_df.head()

,longitudes,latitudes
0,43.781245,-79.432890
1,43.645896,-79.561664
2,43.759272,-79.507922
3,43.757712,-79.235263
4,43.731745,-79.267787


In [24]:
target_coordinates = "43.6532 , -79.3832"
target_search = ("Bar","Pub")
target_radius = 140000
#the radius of toronto is 21 kms
target_type = ("Bar","Pub")
result_list=[]
# set up a parameters dictionary
params = {
    "location": target_coordinates,
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": api_key
    }

# base url
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
response=requests.get(base_url, params=params)


In [25]:
flag = True
count=0
while flag:
    count+=1
    print("Working on Page ",count)

    #response=requests.get(base_url, params=params)
    places_data = response.json()
    lat=[]
    lng=[]
    results=places_data['results']
    Geo=pd.DataFrame(results)['geometry']
    name=pd.DataFrame(results)['name']
    for x in Geo:
        location=pd.DataFrame(x)['location']
        lat.append(location['lat'])
        lng.append(location['lng'])
    

    latitude=pd.DataFrame(lat)
    longitude=pd.DataFrame(lng)
    barname=pd.DataFrame(name)
    barname.join(latitude)
    barname['C'] = barname.reset_index().index
    latitude['C'] = latitude.reset_index().index
    longitude['C'] = longitude.reset_index().index
#barname.join(latitude.set_index('C'))
#longitude.join(latitude.set_index('C'))
    sample2=pd.merge(barname,latitude,on='C')
    sample1=pd.merge(sample2,longitude,on='C')
    result_list.append(sample1)

    if('next_page_token' in places_data):
        paramaters = {
        "key": api_key,
        "pagetoken":places_data['next_page_token']
        }
        time.sleep(2)
       # print(paramaters)
        #print(base_url)
        response=requests.get(base_url, paramaters)
    else:
        flag=False

#response = requests.get(base_url, params=params)
result_df = pd.concat(result_list)[['name','0_x','0_y']]

#print(result_df)
result_df2=pd.DataFrame(result_df)
result_df2

Working on Page  1
Working on Page  2
Working on Page  3


,name,0_x,0_y
0,Scruffy Murphy's Irish Pub,43.707987,-79.393011
1,Good Fortune Bar,43.707687,-79.394572
2,Bar Buca,43.707123,-79.395174
3,Dampa,43.707947,-79.393233
4,Fn Function Bar,43.707457,-79.398326
5,SIP Wine Bar & Authentic Neapolitan Pizza,43.709759,-79.398502
6,Duke of Kent,43.707933,-79.398268
7,Zee Grill Seafood Restaurant & Oyster Bar,43.704913,-79.388642
8,McMurphy's,43.709301,-79.385069
9,Kramer's Bar & Grill,43.698613,-79.396484


In [26]:
gmaps.configure(api_key=api_key)

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(drunk_drivers_df,max_intensity=1)
fig.add_layer(heat_layer)
marker_layer = gmaps.symbol_layer(result_df2[["0_x","0_y"]],stroke_opacity=1,fill_opacity=1)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

# Fatal Collisions vs Neighborhood Mean Income

In [27]:
income_df=pd.read_csv('Resources/Population Income Neighborhood.csv')
income_df.head()

,Unnamed: 0,Category,Population,Neighborhood Mean Income,Lat,Long
0,0,Agincourt North,"29,113",52532.74050,43.808053,-79.266502
1,1,Agincourt South-Malvern West,"23,757",50550.17153,43.788009,-79.283882
2,2,Alderwood,"12,054",55229.36660,43.601710,-79.545238
3,3,Annex,"30,526",45009.96767,43.669833,-79.407585
4,4,Banbury-Don Mills,"27,695",52962.59704,43.749115,-79.366359


In [ ]:
#fig_income = gmaps.figure()

#heat_layer2 = gmaps.heatmap_layer(income_df[["Lat","Long"]],max_intensity=3)
#fig_income.add_layer(heat_layer2)
#income_integer = income_df["Neighborhood Mean Income"].astype(int)
#heat_layer2.point_radius = 3
#marker_layer = gmaps.symbol_layer(TPS_df[["Y","X"]],stroke_opacity=0.2,fill_opacity=0.2,scale=income_df["Neighborhood Mean Income"])
#fig_income.add_layer(marker_layer)
#fig_income

# Fatal Collisions vs Neighborhood Income

In [28]:
filepath_TPS = "Resources/Accident_info (1).csv"
TPS_df = pd.read_csv(filepath_TPS)

TPS_df

,Unnamed: 0,LATITUDE,LONGITUDE,DATE,TIME,YEAR,ACCIDENT LANE,District,INTERSECTION,VICTIM,VICTIM AGE,VEHTYPE,DRIVACT,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,Cause,Neighbourh
0,0,43.816245,-79.227590,2012-01-12T05:00:00.000Z,558,2012,Minor Arterial,Scarborough,Intersection,Pedestrian,40 to 44,Other,,Yes,,Yes,,,Rain,Malvern (132)
1,1,43.637614,-79.447742,2013-11-29T05:00:00.000Z,558,2013,Major Arterial,Toronto and East York,Mid-Block,Driver,25 to 29,"Automobile, Station Wagon",Lost control,,,Yes,,,Speeding,South Parkdale (85)
2,2,43.716773,-79.259883,2018-06-14T04:00:00.000Z,558,2018,Major Arterial,Scarborough,Intersection,Motorcycle Driver,55 to 59,Motorcycle,Driving Properly,,,Yes,Yes,,Unknown,Clairlea-Birchmount (120)
3,3,43.821345,-79.316890,2010-04-13T04:00:00.000Z,559,2010,Major Arterial,Scarborough,Intersection,Passenger,70 to 74,Other,,,,Yes,,,Unknown,Steeles (116)
4,4,43.728129,-79.574641,2014-11-09T05:00:00.000Z,559,2014,Major Arterial,Etobicoke York,Intersection,Passenger,20 to 24,,,,,Yes,,,Unknown,Rexdale-Kipling (4)
5,5,43.758048,-79.513431,2014-11-21T05:00:00.000Z,600,2014,Major Arterial,Etobicoke York,Intersection,Pedestrian,60 to 64,,,Yes,,Yes,,,Unknown,Glenfield-Jane Heights (25)
6,6,43.801421,-79.263774,2015-11-24T05:00:00.000Z,603,2015,Major Arterial,Scarborough,Intersection,Pedestrian,25 to 29,,,Yes,,Yes,,,Speeding,Agincourt North (129)
7,7,43.652745,-79.437390,2011-08-16T04:00:00.000Z,606,2011,Laneway,Toronto and East York,"Park, Private Property, Public Lane",Driver,55 to 59,"Automobile, Station Wagon",Other,,,Yes,,,Unknown,Dufferin Grove (83)
8,8,43.699229,-79.587123,2014-05-21T04:00:00.000Z,610,2014,Major Arterial,Etobicoke York,Intersection,Driver,40 to 44,"Automobile, Station Wagon",Driving Properly,,,Yes,,,Unknown,West Humber-Clairville (1)
9,9,43.664164,-79.326206,2015-09-18T04:00:00.000Z,610,2015,Minor Arterial,Toronto and East York,Intersection,Motorcycle Driver,35 to 39,Motorcycle,Lost control,,,,Yes,,Lost control,Greenwood-Coxwell (65)


In [29]:
lats1=[]
lngs1=[]
for index,row in TPS_df.iterrows():
    lats1.append(row[1])
    lngs1.append(row[2])

In [30]:
popinc_df=pd.read_csv('Resources/Population Income Neighborhood.csv')
popinc_df.head()

,Unnamed: 0,Category,Population,Neighborhood Mean Income,Lat,Long
0,0,Agincourt North,"29,113",52532.74050,43.808053,-79.266502
1,1,Agincourt South-Malvern West,"23,757",50550.17153,43.788009,-79.283882
2,2,Alderwood,"12,054",55229.36660,43.601710,-79.545238
3,3,Annex,"30,526",45009.96767,43.669833,-79.407585
4,4,Banbury-Don Mills,"27,695",52962.59704,43.749115,-79.366359


In [ ]:
# a=[]
# for z in range(0,len(popinc_df['Neighborhood Mean Income'])):
#     a.append((popinc_df.iloc[z,3]/10000).astype(int))
    
# x=(np.array(a)*2).astype(int)
# len(x)
# b=x.tolist()

In [ ]:
# fig = gmaps.figure()

# heat_layer = gmaps.heatmap_layer(TPS_df[["LATITUDE","LONGITUDE"]],max_intensity=1)

# fig.add_layer(heat_layer)

# marker_layer = gmaps.symbol_layer(popinc_df[["Lat","Long"]],stroke_opacity=0.4,fill_opacity=0.6,scale=b)# scale is income parameter
# fig.add_layer(marker_layer)
# fig

In [31]:
from math import sin, cos, sqrt, atan2, radians
def calculate_distance(lat1, lon1, lat2, lon2):
   R = 6373.0
   lat1 = radians(lat1)
   lon1 = radians(lon1)
   lat2 = radians(lat2)
   lon2 = radians(lon2)
   dlon = lon2 - lon1
   dlat = lat2 - lat1
   a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
   c = 2 * atan2(sqrt(a), sqrt(1 - a))
   return R * c
print(calculate_distance(52.2296756,21.0122287,52.406374,16.9251681))

278.54558935106695


In [32]:
popinc_df["Number of Accidents"]=0
popinc_df.head()

,Unnamed: 0,Category,Population,Neighborhood Mean Income,Lat,Long,Number of Accidents
0,0,Agincourt North,"29,113",52532.74050,43.808053,-79.266502,0
1,1,Agincourt South-Malvern West,"23,757",50550.17153,43.788009,-79.283882,0
2,2,Alderwood,"12,054",55229.36660,43.601710,-79.545238,0
3,3,Annex,"30,526",45009.96767,43.669833,-79.407585,0
4,4,Banbury-Don Mills,"27,695",52962.59704,43.749115,-79.366359,0


In [33]:
popinc_df=popinc_df.drop(axis='columns',columns=["Unnamed: 0"])

In [34]:
popinc_df=popinc_df.set_index('Category')

In [35]:
popinc_df.head()

,Population,Neighborhood Mean Income,Lat,Long,Number of Accidents
Category,,,,,
Agincourt North,"29,113",52532.74050,43.808053,-79.266502,0
Agincourt South-Malvern West,"23,757",50550.17153,43.788009,-79.283882,0
Alderwood,"12,054",55229.36660,43.601710,-79.545238,0
Annex,"30,526",45009.96767,43.669833,-79.407585,0
Banbury-Don Mills,"27,695",52962.59704,43.749115,-79.366359,0


In [36]:
for w in range(len(TPS_df)):
    mini=100000
    for y in range(len(popinc_df)):
           
        dist = calculate_distance(TPS_df.iloc[w,1],TPS_df.iloc[w,2],popinc_df.iloc[y,2],popinc_df.iloc[y,3])
        if (dist<mini):
            mini=dist
            y1=y
    popinc_df.iloc[y1,4]=popinc_df.iloc[y1,4]+1
popinc_df.head()

,Population,Neighborhood Mean Income,Lat,Long,Number of Accidents
Category,,,,,
Agincourt North,"29,113",52532.74050,43.808053,-79.266502,8
Agincourt South-Malvern West,"23,757",50550.17153,43.788009,-79.283882,5
Alderwood,"12,054",55229.36660,43.601710,-79.545238,1
Annex,"30,526",45009.96767,43.669833,-79.407585,1
Banbury-Don Mills,"27,695",52962.59704,43.749115,-79.366359,1


In [37]:
plt.scatter(popinc_df['Neighborhood Mean Income'],popinc_df['Number of Accidents'],marker='o',facecolors="green",edgecolors="brown")
plt.xlabel('Neighborhood Mean Income($)')
plt.ylabel('Number of Accidents')
plt.title('Accidents vs Neighborhood Income')
plt.savefig('Accidents vs Neighborhood Income')

<IPython.core.display.Javascript object>

# Type of Vehicle involved in Fatal Collisions

In [39]:
filepath_TPS_V = "Resources/Fatal_Collisions.csv"
TPS_V_df = pd.read_csv(filepath_TPS_V)
a=TPS_V_df[TPS_V_df['IMPACTYPE']=='Pedestrian Collisions']
a

,X,Y,Index_,ACCNUM,YEAR,DATE,TIME,Hour,STREET1,STREET2,...,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,Hood_ID,Neighbourh,ObjectId
0,-79.227590,43.816245,7377669,1275814,2012,2012-01-12T05:00:00.000Z,558,5,NEILSON RD,FINCH AVE E,...,,Yes,,Yes,,,,132,Malvern (132),1
5,-79.513431,43.758048,8000170,143339762,2014,2014-11-21T05:00:00.000Z,600,6,FINCH AVE W,DRIFTWOOD AVE,...,,,,,,,,25,Glenfield-Jane Heights (25),6
6,-79.263774,43.801421,80493406,5002013764,2015,2015-11-24T05:00:00.000Z,603,6,MCCOWAN RD,KENHATCH BLVD,...,,,Yes,Yes,Yes,,,129,Agincourt North (129),7
11,-79.291316,43.765433,8000333,5000160956,2015,2015-01-28T05:00:00.000Z,617,6,1792 BIRCHMOUNT RD,,...,,,,,,,,126,Dorset Park (126),12
12,-79.379863,43.655387,80000046,142879736,2014,2014-09-10T04:00:00.000Z,620,6,OKEEFE LANE,DUNDAS SQ,...,,,,,,,,75,Church-Yonge Corridor (75),13
14,-79.469287,43.637206,80973076,8001812203,2018,2018-10-01T04:00:00.000Z,622,6,THE QUEENSWAY,WINDERMERE AVE,...,,,,,,,,87,High Park-Swansea (87),15
15,-79.394183,43.675101,80985326,8002341194,2018,2018-12-22T05:00:00.000Z,623,6,DAVENPORT RD,HAZELTON AVE,...,,,Yes,Yes,,,,95,Annex (95),16
16,-79.276990,43.702545,5808587,1138618,2009,2009-11-24T05:00:00.000Z,630,6,WARDEN AVE,CATARAQUI CRES,...,,,,,,,,120,Clairlea-Birchmount (120),17
17,-79.394990,43.645445,6689759,1260146,2011,2011-10-21T04:00:00.000Z,630,6,SPADINA AVE W,KING ST W,...,,Yes,,,,,,77,Waterfront Communities-The Island (77),18
19,-79.450557,43.639189,80489687,5001649654,2015,2015-09-24T04:00:00.000Z,636,6,THE QUEENSWAY,GLENDALE AVE,...,,,,,,,,87,High Park-Swansea (87),20


In [20]:
lats_v=[]
lngs_v=[]
for index,row in TPS_V_df.iterrows():
   lats_v.append(row[0])
   lngs_v.append(row[1])
    
fig_v = gmaps.figure()


symbol_layer=gmaps.symbol_layer(TPS_V_df[["Y","X"]],stroke_opacity=0.1,fill_opacity=0.1)
fig_v.add_layer(symbol_layer)
fig_v

Figure(layout=FigureLayout(height='420px'))

# Cause of Collision

In [40]:
filepath_coll = "Resources/Fatal_Collisions.csv"
fatal_coll=pd.read_csv(filepath_coll)
fatal_coll.head()

,X,Y,Index_,ACCNUM,YEAR,DATE,TIME,Hour,STREET1,STREET2,...,EMERG_VEH,PASSENGER,SPEEDING,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,Hood_ID,Neighbourh,ObjectId
0,-79.227590,43.816245,7377669,1275814,2012,2012-01-12T05:00:00.000Z,558,5,NEILSON RD,FINCH AVE E,...,,Yes,,Yes,,,,132,Malvern (132),1
1,-79.447742,43.637614,80000014,131058335,2013,2013-11-29T05:00:00.000Z,558,5,1491 LAKE SHORE BLVD W,,...,,,Yes,Yes,,,,85,South Parkdale (85),2
2,-79.259883,43.716773,80944448,8001077165,2018,2018-06-14T04:00:00.000Z,558,5,ST CLAIR AVE E,DANFORTH RD,...,,,,Yes,,,,120,Clairlea-Birchmount (120),3
3,-79.316890,43.821345,6253540,1162012,2010,2010-04-13T04:00:00.000Z,559,5,STEELES AVE E,BIRCHMOUNT RD,...,,Yes,,Yes,Yes,,Yes,116,Steeles (116),4
4,-79.574641,43.728129,80501551,4003267619,2014,2014-11-09T05:00:00.000Z,559,5,KIPLING AVE,WESTHUMBER BLVD,...,,Yes,,Yes,Yes,,,4,Rexdale-Kipling (4),5


In [41]:
fatal_col1=fatal_coll[['LATITUDE','LONGITUDE','STREET1','STREET2','DATE','TIME','YEAR','ROAD_CLASS','District','LOCCOORD','TRAFFCTL','VISIBILITY','LIGHT',
                       'INVTYPE','INVAGE','VEHTYPE','MANOEUVER','DRIVACT','DRIVCOND','PEDESTRIAN','CYCLIST','AUTOMOBILE','MOTORCYCLE','TRUCK','SPEEDING','REDLIGHT','ALCOHOL','Neighbourh','ObjectId']]

In [42]:
fatal_col1.columns

Index(['LATITUDE', 'LONGITUDE', 'STREET1', 'STREET2', 'DATE', 'TIME', 'YEAR',
       'ROAD_CLASS', 'District', 'LOCCOORD', 'TRAFFCTL', 'VISIBILITY', 'LIGHT',
       'INVTYPE', 'INVAGE', 'VEHTYPE', 'MANOEUVER', 'DRIVACT', 'DRIVCOND',
       'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK',
       'SPEEDING', 'REDLIGHT', 'ALCOHOL', 'Neighbourh', 'ObjectId'],
      dtype='object')

In [43]:
fatal_col2=fatal_col1.rename(columns={"LOCCOORD":"INTERSECTION","ROAD_CLASS":"ACCIDENT LANE","INVTYPE":"VICTIM","INVAGE":"VICTIM AGE"})
fatal_col2.columns

Index(['LATITUDE', 'LONGITUDE', 'STREET1', 'STREET2', 'DATE', 'TIME', 'YEAR',
       'ACCIDENT LANE', 'District', 'INTERSECTION', 'TRAFFCTL', 'VISIBILITY',
       'LIGHT', 'VICTIM', 'VICTIM AGE', 'VEHTYPE', 'MANOEUVER', 'DRIVACT',
       'DRIVCOND', 'PEDESTRIAN', 'CYCLIST', 'AUTOMOBILE', 'MOTORCYCLE',
       'TRUCK', 'SPEEDING', 'REDLIGHT', 'ALCOHOL', 'Neighbourh', 'ObjectId'],
      dtype='object')

In [44]:
fatal_col3=fatal_col2[['LATITUDE','LONGITUDE','DATE','TIME','YEAR','ACCIDENT LANE','District','INTERSECTION','VISIBILITY','LIGHT',
                       'VICTIM','VICTIM AGE','VEHTYPE','DRIVACT','PEDESTRIAN','CYCLIST','AUTOMOBILE','MOTORCYCLE','TRUCK','SPEEDING','REDLIGHT','ALCOHOL','Neighbourh','ObjectId']]
fatal_col3.columns

Index(['LATITUDE', 'LONGITUDE', 'DATE', 'TIME', 'YEAR', 'ACCIDENT LANE',
       'District', 'INTERSECTION', 'VISIBILITY', 'LIGHT', 'VICTIM',
       'VICTIM AGE', 'VEHTYPE', 'DRIVACT', 'PEDESTRIAN', 'CYCLIST',
       'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'SPEEDING', 'REDLIGHT', 'ALCOHOL',
       'Neighbourh', 'ObjectId'],
      dtype='object')

In [45]:
fatal_col4=fatal_col3
fatal_col4.columns

Index(['LATITUDE', 'LONGITUDE', 'DATE', 'TIME', 'YEAR', 'ACCIDENT LANE',
       'District', 'INTERSECTION', 'VISIBILITY', 'LIGHT', 'VICTIM',
       'VICTIM AGE', 'VEHTYPE', 'DRIVACT', 'PEDESTRIAN', 'CYCLIST',
       'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'SPEEDING', 'REDLIGHT', 'ALCOHOL',
       'Neighbourh', 'ObjectId'],
      dtype='object')

In [46]:
def cause_finder(r):
    
    if ((r['ALCOHOL'] == 'Yes') and (r['SPEEDING'] == 'Yes')):
        v = "Alcohol & Speeding"
    elif (r['SPEEDING'] == 'Yes'):
        v = "Speeding"
    elif (r['ALCOHOL'] == 'Yes'):
        v = "Alcohol"
    elif ((r['DRIVACT']=='Disobeyed Traffic Control') or (r['DRIVACT']=='Exceeding Speed Limit') or 
          (r['DRIVACT']=='Failed to Yield Right of Way') or (r['DRIVACT']=='Following too Close') or 
         (r['DRIVACT']=='Improper Lane Change') or (r['DRIVACT']=='Improper Passing') or
         (r['DRIVACT']=='Improper Turn') or (r['DRIVACT']=='Lost control') or (r['DRIVACT']=='Speed too Fast For Condition') or
         (r['DRIVACT']=='Wrong Way on One Way Road') or (r['DRIVACT']=='Speed too Slow')):
        v= r['DRIVACT']
    elif ((r['VISIBILITY']=='Rain') or (r['VISIBILITY']=='Snow') or (r['VISIBILITY']=='Freezing Rain') or
          (r['VISIBILITY']=='Fog, Mist, Smoke, Dust')):
        v= r['VISIBILITY']
    else:
        v='Unknown'
 
    return v

fatal_col4['Cause'] = fatal_col4.apply(cause_finder, axis = 1)
#fatal_col4.loc[fatal_col4['Cause_Real']=='']
fatal_col4['Cause'].value_counts()

/Users/khaledkhatib/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Unknown                         350
Speeding                        126
Rain                             51
Lost control                     30
Alcohol & Speeding               20
Alcohol                          10
Disobeyed Traffic Control         8
Snow                              6
Failed to Yield Right of Way      3
Improper Turn                     2
Improper Lane Change              1
Following too Close               1
Speed too Slow                    1
Freezing Rain                     1
Name: Cause, dtype: int64

In [47]:
fatal_col4.head()

,LATITUDE,LONGITUDE,DATE,TIME,YEAR,ACCIDENT LANE,District,INTERSECTION,VISIBILITY,LIGHT,...,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,SPEEDING,REDLIGHT,ALCOHOL,Neighbourh,ObjectId,Cause
0,43.816245,-79.227590,2012-01-12T05:00:00.000Z,558,2012,Minor Arterial,Scarborough,Intersection,Rain,"Dark, artificial",...,,Yes,,,,,,Malvern (132),1,Rain
1,43.637614,-79.447742,2013-11-29T05:00:00.000Z,558,2013,Major Arterial,Toronto and East York,Mid-Block,Clear,"Dawn, artificial",...,,Yes,,,Yes,,,South Parkdale (85),2,Speeding
2,43.716773,-79.259883,2018-06-14T04:00:00.000Z,558,2018,Major Arterial,Scarborough,Intersection,Clear,Daylight,...,,Yes,Yes,,,,,Clairlea-Birchmount (120),3,Unknown
3,43.821345,-79.316890,2010-04-13T04:00:00.000Z,559,2010,Major Arterial,Scarborough,Intersection,Clear,Dawn,...,,Yes,,,,Yes,,Steeles (116),4,Unknown
4,43.728129,-79.574641,2014-11-09T05:00:00.000Z,559,2014,Major Arterial,Etobicoke York,Intersection,Clear,"Dark, artificial",...,,Yes,,,,Yes,,Rexdale-Kipling (4),5,Unknown


In [48]:
Accident_info=fatal_col4[['LATITUDE', 'LONGITUDE', 'DATE', 'TIME', 'YEAR', 'ACCIDENT LANE',
       'District', 'INTERSECTION','VICTIM',
       'VICTIM AGE', 'VEHTYPE', 'DRIVACT', 'PEDESTRIAN', 'CYCLIST',
       'AUTOMOBILE', 'MOTORCYCLE', 'TRUCK', 'Cause','Neighbourh']]
Accident_info.to_csv("Accident_info.csv")
fatal_col4.to_csv("Resources/Fatal Collisions.csv")
Accident_info.head()

,LATITUDE,LONGITUDE,DATE,TIME,YEAR,ACCIDENT LANE,District,INTERSECTION,VICTIM,VICTIM AGE,VEHTYPE,DRIVACT,PEDESTRIAN,CYCLIST,AUTOMOBILE,MOTORCYCLE,TRUCK,Cause,Neighbourh
0,43.816245,-79.227590,2012-01-12T05:00:00.000Z,558,2012,Minor Arterial,Scarborough,Intersection,Pedestrian,40 to 44,Other,,Yes,,Yes,,,Rain,Malvern (132)
1,43.637614,-79.447742,2013-11-29T05:00:00.000Z,558,2013,Major Arterial,Toronto and East York,Mid-Block,Driver,25 to 29,"Automobile, Station Wagon",Lost control,,,Yes,,,Speeding,South Parkdale (85)
2,43.716773,-79.259883,2018-06-14T04:00:00.000Z,558,2018,Major Arterial,Scarborough,Intersection,Motorcycle Driver,55 to 59,Motorcycle,Driving Properly,,,Yes,Yes,,Unknown,Clairlea-Birchmount (120)
3,43.821345,-79.316890,2010-04-13T04:00:00.000Z,559,2010,Major Arterial,Scarborough,Intersection,Passenger,70 to 74,Other,,,,Yes,,,Unknown,Steeles (116)
4,43.728129,-79.574641,2014-11-09T05:00:00.000Z,559,2014,Major Arterial,Etobicoke York,Intersection,Passenger,20 to 24,,,,,Yes,,,Unknown,Rexdale-Kipling (4)


In [49]:
Cause_df=Accident_info.groupby('Cause').count()
Cause=Cause_df.drop(axis='index',index='Unknown')

In [52]:
plt.barh(Cause.index,width=Cause['DRIVACT'],height=0.8,color="green")
plt.xlabel('Number of Accidents')
plt.ylabel('Cause')
plt.title('Accidents vs Cause')
plt.savefig('Accidents vs Cause.PNG')